In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

from datafold.appfold import EDMDControl
from datafold.dynfold.dmd import ControlledAffineDynamicalSystem, gDMDAffine
from datafold.dynfold.transform import TSCPolynomialFeatures, TSCRadialBasis
from datafold.pcfold import InitialCondition, InverseQuadraticKernel, TSCDataFrame
from datafold.utils._systems import InvertedPendulum

### Random system of the form `xdot = A @ x + B @ u @ x`

In [ ]:
state_size = 4
input_size = 2
n_timesteps = 50
n_ic = 1

gen = np.random.default_rng(22)

A = gen.uniform(-0.5, 0.5, size=(state_size, state_size))
np.fill_diagonal(A, gen.uniform(-1.0, -0.5, size=state_size))
x0 = gen.uniform(-1.0, 1.0, size=(state_size, n_ic))
x0 = np.hstack([x0, x0, x0, x0])
Bi = np.stack(
    [gen.uniform(-0.5, 0.5, size=(state_size, state_size)) for i in range(input_size)],
    2,
)
u = np.concatenate(
    [
        np.ones((n_ic, n_timesteps, input_size)),
        -np.ones((n_ic, n_timesteps, input_size)),
        np.stack([np.ones((n_ic, n_timesteps)), -np.ones((n_ic, n_timesteps))], 2),
        np.stack([-np.ones((n_ic, n_timesteps)), np.ones((n_ic, n_timesteps))], 2),
    ],
    0,
)
t = np.linspace(0, n_timesteps - 1, n_timesteps) * 0.1
names = ["x" + str(i + 1) for i in range(state_size)]

tsc_df = (
    ControlledAffineDynamicalSystem()
    .setup_matrix_system(A, Bi)
    .evolve_system(x0, u, time_values=t, time_delta=0.1, feature_names_out=names)
)
# print(tsc_df)
# print(A)
# print(Bi)
ureshaped = u.reshape((-1, input_size))
for i in range(input_size):
    tsc_df["u" + str(i + 1)] = ureshaped[:, i]
tsc_df.plot()

In [ ]:
dmd = gDMDAffine().fit(tsc_df, split_by="name", control=["u1", "u2"])

In [ ]:
tsc_pred = dmd.fit_predict(tsc_df)
tsc_pred.plot()

In [ ]:
(tsc_df - tsc_pred).plot()

### Duffing

In [ ]:
def duffing(t, x, alpha=-1, beta=1, delta=0.6, u=lambda t: 1):
    xdot = np.array([x[1], 0])
    xdot[1] = -delta * x[1] - alpha * x[0] - beta * x[0] ** 3 + u(t)
    return xdot


def simulate(x0, u=lambda t: 1, N=201, t0=0, tf=2):
    sol = solve_ivp(
        lambda t, x: duffing(t, x, u=u), (t0, tf), x0, t_eval=np.linspace(t0, tf, N)
    )
    if not sol.success:
        raise RuntimeError("Couldn't not evolve the system.")
    df = pd.DataFrame(data=sol.y.T, index=sol.t, columns=["x", "xdot"])
    df["u"] = u(sol.t)
    return df


x0 = np.array([0.1, -0.9])
x = simulate(x0, lambda t: -1)
plt.plot(x["x"])
x = simulate(x0)
plt.plot(x["x"])
x = simulate(x0, lambda t: np.sin(np.pi * t))
plt.plot(x["x"], "purple")
x = simulate(x0, lambda t: 0)
plt.plot(x["x"], "gold")

In [ ]:
n_repeats = 100
rng = np.random.default_rng(0)
dflist = []
for i in range(n_repeats):
    random_x0 = rng.uniform(-3, 3, 2)
    dflist.append(simulate(random_x0, N=21))
    dflist.append(simulate(random_x0, lambda t: -1, N=21))
duffing_tsc = TSCDataFrame.from_frame_list(dflist)
duffing_tsc.plot()

In [ ]:
edmd_duffing = EDMDControl(
    dict_steps=[
        ("poly", TSCPolynomialFeatures(5, include_bias=True)),
    ],
    dmd_model=gDMDAffine(rcond=1e-6),
)
edmd_duffing.fit(duffing_tsc, split_by="name", control=["u"]);

In [ ]:
plt.figure(figsize=(8, 3))
plt.subplot(121)
plt.imshow(edmd_duffing.sys_matrix)
plt.colorbar()
plt.title("System matrix")
print("Sys eigenvalues: ", np.linalg.eigvals(edmd_duffing.sys_matrix))
plt.subplot(122)
plt.imshow(edmd_duffing.control_matrix[:, :, 0])
plt.colorbar()
plt.title("Control matrix")
print("Control eigenvalues: ", np.linalg.eigvals(edmd_duffing.control_matrix[:, :, 0]))

In [ ]:
n_samples = 50
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=np.linspace(0, 2, n_samples),
    control_input=-np.ones(n_samples),
)
plt.plot(pred0["x"].values)
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=np.linspace(0, 2, n_samples),
    control_input=np.ones(n_samples),
)
plt.plot(pred0["x"].values)
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=np.linspace(0, 2, n_samples),
    control_input=np.sin(np.pi * np.linspace(0, 1, n_samples)),
)
plt.plot(pred0["x"].values, "purple")
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=np.linspace(0, 2, n_samples),
    control_input=0 * np.ones(n_samples),
)
plt.plot(pred0["x"].values, "gold")

### Inverted Pendulum

In [ ]:
state_cols = ["x", "xdot", "theta", "thetadot"]
control_cols = ["u"]

# Data generation parameters
sim_time_step = 0.01  # s
sim_num_steps = 1000  # -
training_size = 20  # -
ic = InitialCondition.from_array(np.array([0, 0, np.pi, 0]), columns=state_cols)

invertedPendulum = InvertedPendulum(initial_condition=ic.values)

Xlist, Ulist = [], []
np.random.seed(42)
for i in range(training_size):
    control_amplitude = 0.1 + 0.9 * np.random.random()
    control_frequency = np.pi + 2 * np.pi * np.random.random()
    control_phase = 2 * np.pi * np.random.random()
    control_func = lambda t, y: control_amplitude * np.sin(
        control_frequency * t + control_phase
    )
    invertedPendulum.reset()
    traj = invertedPendulum.predict(
        time_step=sim_time_step,
        num_steps=sim_num_steps,
        control_func=control_func,
    )
    assert (
        invertedPendulum.sol.success
    ), f"Divergent solution for amplitude={control_amplitude}, frequency={control_frequency}"
    t = invertedPendulum.sol.t
    dfx = pd.DataFrame(data=traj.T, index=t, columns=state_cols)
    dfx[control_cols] = 0.0
    Xlist.append(dfx)
    control_input = control_func(t, traj)
    dfu = pd.DataFrame(data=control_input, index=t, columns=control_cols)
    for col in state_cols:
        dfu[col] = 0.0
    dfu = dfu[state_cols + control_cols]
    Ulist.append(dfu)

X_tsc = TSCDataFrame.from_frame_list(Xlist)[state_cols]
X_tsc[control_cols] = TSCDataFrame.from_frame_list(Ulist)[control_cols]

In [ ]:
num_rbfs = 20
eps = 1

rbf = TSCRadialBasis(
    kernel=InverseQuadraticKernel(epsilon=eps), center_type="fit_params"
)
center_ids = sorted(
    np.random.choice(
        range(0, sim_num_steps * training_size), size=num_rbfs, replace=False
    )
)
centers = X_tsc.iloc[center_ids].values

In [ ]:
edmdrbf = EDMDControl(
    dict_steps=[
        ("rbf", rbf),
    ],
    include_id_state=True,
    dmd_model=gDMDAffine(rcond=1e-6),
)

edmdrbf.fit(
    X_tsc,
    split_by="name",
    state=state_cols,
    control=control_cols,
    rbf__centers=centers[:, :-1],
)
rbfprediction = edmdrbf.predict(
    ic, control_input=np.atleast_2d(control_input).T, time_values=t
)
plt.figure(figsize=(16, 3))
plt.subplot(121)
plt.plot(rbfprediction["x"].values[:100], label="prediction")
plt.plot(dfx["x"].values[:100], label="actual")
plt.legend()
plt.title(r"EDMD(100 random rbf) prediction - cart position $x$")
plt.subplot(122)
plt.plot(rbfprediction["theta"].values[:100], label="prediction")
plt.plot(dfx["theta"].values[:100], label="actual")
plt.legend()
plt.title(r"EDMD(100 random rbf) prediction - pendulum angle $\theta$");

In [ ]:
edmdrbf.predict(
    ic, control_input=np.atleast_2d(control_input[:10]).T, time_values=t[:10]
)

### Einsum test

In [ ]:
np.einsum(
    "ij,ik->ijk",
    np.array([[1, -1], [2, -2], [3, -3]]),
    np.array([[0.1, 0.2, 0.3, 0.4], [1.1, 1.2, 1.3, 1.4], [10.1, 12.1, 13.1, 14.1]]),
).reshape(3, 2 * 4)

In [ ]:
np.einsum(
    "ij,ik->ijk",
    np.array([[0.1, 0.2, 0.3, 0.4], [1.1, 1.2, 1.3, 1.4], [10.1, 12.1, 13.1, 14.1]]),
    np.array([[1, -1], [2, -2], [3, -3]]),
).reshape(3, 2 * 4)